In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("preprocessed_smiles.csv",header=None)

In [4]:
text = "".join(df[1])
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))# cutting the SMILES corpus into the sequences used for training
maxlen = 40
step = 3
text = "".join(df[1][:1000000])

corpus length: 96434192
total chars: 47


In [5]:
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 30777
Vectorization...


In [6]:
X.shape

(30777, 40, 47)

In [7]:
y.shape

(30777, 47)

In [37]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [39]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]),return_sequences=True,recurrent_dropout=0.2))
model.add(BatchNormalization(axis=1))
model.add(LSTM(256,return_sequences=False,recurrent_dropout=0.2))
model.add(BatchNormalization(axis=1))
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
                  metrics=['accuracy'])


In [42]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
early_stop = keras.callbacks.EarlyStopping(monitor='loss', verbose=1, patience=7)
callbacks_list = [checkpoint]

In [43]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
30777/30777 [==============================] - 287s 9ms/step - loss: 1.3598 - acc: 0.5587

Epoch 00001: loss improved from inf to 1.35982, saving model to weights-improvement-01-1.3598.hdf5
Epoch 2/20
30777/30777 [==============================] - 294s 10ms/step - loss: 1.2263 - acc: 0.5946

Epoch 00002: loss improved from 1.35982 to 1.22631, saving model to weights-improvement-02-1.2263.hdf5
Epoch 3/20
30777/30777 [==============================] - 285s 9ms/step - loss: 1.1294 - acc: 0.6216

Epoch 00003: loss improved from 1.22631 to 1.12943, saving model to weights-improvement-03-1.1294.hdf5
Epoch 4/20
30777/30777 [==============================] - 285s 9ms/step - loss: 1.0627 - acc: 0.6409

Epoch 00004: loss improved from 1.12943 to 1.06265, saving model to weights-improvement-04-1.0627.hdf5
Epoch 5/20
30777/30777 [==============================] - 298s 10ms/step - loss: 0.9930 - acc: 0.6594

Epoch 00005: loss improved from 1.06265 to 0.99304, saving model to weights-impr

In [44]:
# pick a random seed
start = np.random.randint(0, len(X)-1)
pattern = X[start]
print ("Seed:")
print ("\"", ''.join([indices_char[value.argmax()] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern,(1,pattern.shape[0],pattern.shape[1]))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = indices_char[index]
    seq_in = [indices_char[value.argmax()]for value in pattern]
    print(result)
    row = np.zeros(len(chars))
    row[index] = 1
    pattern = np.vstack([pattern,row])
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" 5ccccc5CC12CCC3C(CCCCA(Cc4ccccc4)C5=CC(= "
O
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O
)
c
3
A
A
c
4
c
c
c
c
c
3
4
C
C
N
(
C
C
=
C
(
C
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
)
N
C
(
C
)
C
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O
)
c
3
A
A
c
4
c
c
c
c
c
3
4
C
C
N
(
C
C
=
C
(
C
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
)
N
C
(
C
)
C
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O
)
c
3
A
A
c
4
c
c
c
c
c
3
4
C
C
N
(
C
C
=
C
(
C
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
)
N
C
(
C
)
C
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O
)
c
3
A
A
c
4
c
c
c
c
c
3
4
C
C
N
(
C
C
=
C
(
C
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
)
N
C
(
C
)
C
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O
)
c
3
A
A
c
4
c
c
c
c
c
3
4
C
C
N
(
C
C
=
C
(
C
)
C
C
C
C
C
C
C
C
C
C
C
C
C
C
O
)
N
C
(
C
)
C
c
1
c
c
c
c
c
1
C
(
=
O
)
N
C
C
C
A
(
C
c
2
c
c
c
c
c
2
)
C
(
=
O

In [32]:
pattern.shape

(40, 47)

In [45]:
chars

['#',
 '%',
 '(',
 ')',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '=',
 'A',
 'B',
 'C',
 'F',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'V',
 'X',
 'Z',
 'a',
 'b',
 'c',
 'e',
 'g',
 'i',
 'l',
 'o',
 'p',
 'r',
 's',
 't',
 'u']